# Accesso parametrico a Course Catalogue

Gli indirizzi web delle pagine degli insegnamenti su CC contengono codici complicati da prevedere, che dipendono dagli anni, dai pian di studio e dall'esame stesso. Da GDA si evince che esite una modalità di accesso parametrica, anche se si nota poco perché genera subito un redirect sugli indirizzi di cui sopra.

```http
https://unipi.coursecatalogue.cineca.it/af/2024?
    corso=WFI-LM&
    annoOrdinamento=2023&
    ad=217BB
```

dove ci sono i seguenti **parametri obbligatori**

* dopo `af` (attività formativa?) va indicato l'anno di erogazione
* `corso` specifica il corso di laurea
* `annoOrdinamento` specifica l'ordinamento attivo, questo va conosciuto o ricavato dalle API.
* `ad` (attività didattica?) indica il codice ESSE3

oltre a questo volendo ci sono i seguenti **parametri opzionali**

* `pds` (piano di studi)... qui va conosciuto il codice di GDA, ma si può ricavare dalle API
* `coorte` anno della coorte
* ... altri parametri direi inutili

Se i parametri opzionali non sono specificati, sceglie CC, non so bene in base a cosa.

## Esempi

* **204BB** erogato in AA2024/25 nel CV di Materia per gli studenti immatricolati nel 2024/25

  &rarr; Indirizzo parametrico: [https://unipi.coursecatalogue.cineca.it/af/2024?corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=9&coorte=2024](https://unipi.coursecatalogue.cineca.it/af/2024?corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=9&coorte=2024)
  
  &rarr; Redirect: [https://unipi.coursecatalogue.cineca.it/insegnamenti/2024/52568_686730_71640/2023/52568/10452?annoOrdinamento=2023&coorte=2024](https://unipi.coursecatalogue.cineca.it/insegnamenti/2024/52568_686730_71640/2023/52568/10452?annoOrdinamento=2023&coorte=2024)

* **204BB** erogato in AA2024/25 nel CV di Astrofisica per gli studenti immatricolati nel 2023/24

  &rarr; Indirizzo parametrico: [https://unipi.coursecatalogue.cineca.it/af/2024?corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=11&coorte=2023](https://unipi.coursecatalogue.cineca.it/af/2024?corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=11&coorte=2023)

  &rarr; Redirect: [https://unipi.coursecatalogue.cineca.it/insegnamenti/2024/51377_686730_71640/2023/51377/10452?annoOrdinamento=2023&coorte=2023](https://unipi.coursecatalogue.cineca.it/insegnamenti/2024/51377_686730_71640/2023/51377/10452?annoOrdinamento=2023&coorte=2023)

## Chiamate API su Course Catalogue

CC espone varie API con un endpoint pubblico su web. Le richieste sono dei GET o POST a seconda dei casi, con alcuni parametri. La risposta è un JSON.

### 1. Info sul corso di laurea con `api/v1/corso`

La seguente sintassi produce un JSON abbastanza enorme con tutti gli insegnamenti e i piani di studio

[https://unipi.coursecatalogue.cineca.it/api/v1/corso/2024/10452](https://unipi.coursecatalogue.cineca.it/api/v1/corso/2024/10452)

dove 2024 è l'anno di offerta e 10452 è il codice GDA del corso di studi. Volendo ci sono anche dei parametri ma sono inutili mi pare.

Cosa è possibile estrarre dal JSON? Sostanzialmente tutte (e altre) le info che si trovano quando si accede su CC alla pagina di un corso di laurea. La struttura JSON è parecchio incasinata, basta cliccare il link web sopra per vederla... sono enne pagine. Se la chiamo `data`:

* sorvolo su descrizioni varie, requisiti d'accesso ecc...
* `data.cdsCode` è il solito WFI-LM
* `data.des_it` è FISICA
* `data.ordinamento_aa` è l'anno dell'ordinamento attivo (quello che va indicato nell'indirizzo parametrico sopra)
* `data.percorsi` è un array, dove "percorsi" = CURRICULA da cui si estrae
  * `data.percorsi[n].pdsId` il codice del pds che entra negli indirizi web di CC
  * `data.percorsi[n].pdsCod` il codice (in genere un numero) che va usato nell'indirizzo parametrico dell'insegnamento indicato sopra
  * `data.percorsi[n].des_it` il nome del CURRICULUM
  * varie sottostrutture.. 
    * ogni percorso contiene un array `anni`
      * ogni anno a sua volta contiene un array `insegnamenti`, che di fatto sono gruppi di esami
        * per ogni gruppo di esami ci sono un tot di cose, nome, ecc...
        * ogni gruppo contiene un array di `attivita` che... sono gli insegnamenti ¯\\\_(ツ)\_/¯
          * detto `att` il singolo elemento dell'ultimo array...
            * `att.adCod` è il codice ESSE3 del corso
            * `att.cod` è il codice misterioso di CC... che nessuno riesce mai a calcolare. Esempio: 52567_686748_74861
            * `att.crediti`
            * `att.ore`
            * `att.periodo_idattico_it`
            * `att.des_it` il nome dell'insegnamento
            * `att.tafDes_it` Caratterizzante / Affine?
            * `att.docenti` è un vettore che FORSE contiene la lista dei docenti... con tanto di matricola UniPi
            * ecc ecc

> **Nota sul sovraccarico web di queste operazioni.** Inesistente... mi pare che ogni volta che si apre la pagina di un corso di laurea Chrome ricarica questo malloppo, talvolta anche più di una volta sola.
  
### 2. Info su specifico esame con `api/v1/attivitaFormativa`

Questa API non è del tutto inutile rispetto a quella precedente e restituisce delle informazioni in più, come il contenuto del corso, o altre info varie. Le informazioni che vanno fornite sono le stesse che andrebbero fornite per l'accesso parametrico al corso indicato all'inizio

Esempio: per saperne di più su **204BB** erogato in AA2024/25 nel CV di Astrofisica per gli studenti immatricolati nel 2023/24, basta interrogare

  [https://unipi.coursecatalogue.cineca.it/api/v1/attivitaFormativa?aaOfferta=2024&corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=11&coorte=2023](https://unipi.coursecatalogue.cineca.it/api/v1/attivitaFormativa?aaOfferta=2024&corso=WFI-LM&annoOrdinamento=2023&ad=204BB&pds=11&coorte=2023)

Chiamato `data` il JSON ricevuto in risposta, abbiamo

* `data.des_it` è il titolo 
* `data.corso_percorso_des_it` è il curriculum
* `data.periodo_didattico_it` 
* `data.crediti` 
* `data.ssd` qui fornisce anche SSD
* `data.tipo_it` Caratterizzante o altro...
* `data.ssd` qui fornisce anche SSD
* i docenti sono sparsi in vari arrays 
    * `data.titolari` 
    * `data.responsabili` 
    * `data.docenti`
* esiste un vettore `data.testiTotali`... in cui ci sono i contenuti dell'insegnamento e altre cose. Vedere esempio.

### 3. Altre API

Le due API sopra bastano per fare praticamente tutto... ma ne estono almneo altre due, che permettono di fare direttamente una query sul database, e su cui si appoggiano le pagine web di search di CC. Diversamente dalle due precedenti, qui è necessario fare un POST e non un GET, quindi non è interrogabile direttamente da browser. Riporto solo due esecuzioni in sintassi python:

1. Query insegnamenti `api/v1/ricercaInsegnamenti`  


  
2. Query corsi `api/v1/ricercaInsegnamenti`  

  ```python
      url = "https://unipi.coursecatalogue.cineca.it/api/v1/ricercaCorsi"
      payload = {
        "searchString": "fisica",
        "anno": "2024",
        "tipoCorso": None,
        "sede": None,
        "lingua": None,
        "dipartimento": None,
        "interateneo": None,
        "area": None
      }
      headers = {"Content-Type": "application/json"}
      response = requests.post(url, json=payload, headers=headers)
  ```

> **Warning**. Opportunità e potenziale castastrofe: la query di ricerca con API è più permissiva di quella web. Per esempio è possible cercare *tutti* gli insegnamenti del dipartimento di Fisica... mentre il form web richiede di restringere la ricerca specificando qualcosa nel campo "insegnamento". Forse è possibile farsi elencare tutti gli insegnamenti di tutta UniPi. L'impressione netta è che non ci sia nessun filtro. Non provato. E non proverei.
  


In [ ]:
# Interrogazione api/v1/attivitaFormativa

codiceESSE3 = "204BB"
annoOfferta = 2024
coorte      = 2023
pds         = 11

print(f"Interrogazione: [{codiceESSE3}] offerto {annoOfferta}, coorte {coorte} pds {pds}\n")

import requests
import textwrap

url = "https://unipi.coursecatalogue.cineca.it/api/v1/attivitaFormativa"
params = {
    "aaOfferta": annoOfferta,
    "corso": "WFI-LM",
    "annoOrdinamento": 2023,
    "ad": codiceESSE3,
    "coorte": coorte,
    "pds": pds
}
response = requests.get(url, params=params)

if response.status_code == 200:

    data = response.json()
    print(f"Titolo:   {data['des_it']}")
    print(f"PDS:      {data['corso_percorso_des_it']}")
    print(f"Tipo:     {data['tipo_it']}")
    print(f"SSD:      {data['ssd']}")
    print(f"CFU:      {data['crediti']} CFU")
    print(f"Periodo:  {data['periodo_didattico_it']}")
    print(f"Codice:   {data['cod']}")
    print(f"Contenuti:\n")

    for line in textwrap.wrap(data['testiTotali'][0]['contenuti_it'], 80):
        print(f"    {line}")

    docenti = ['* '+xx['des'] for xx in data['docenti']]
    print(f"\nLista docenti:")
    print("\n".join(docenti))

else:
    print(f"Errore nella richiesta: {response.status_code}")

Interrogazione: [204BB] offerto 2024, coorte 2023 pds 11

Titolo:   SOLID STATE PHYSICS / FISICA DELLO STATO SOLIDO
PDS:      ASTRONOMIA E ASTROFISICA
Tipo:     Caratterizzante
SSD:      FIS/03
CFU:      9 CFU
Periodo:  Primo Ciclo Semestrale
Codice:   51377_686730_71640
Contenuti:

    <p>Stati quantici in un potenziale periodico 1D: teorema di Bloch e soluzioni
    nel limite quasi-libero, del tight-binding e del tunneling risonante. Teorema
    dell&rsquo;accelerazione, quasi-momento, massa effettiva, pittura delle buche.
    Cristalli in 3D: reticolo diretto e reciproco, scattering di Von Laue e di
    Bragg, strutture cristalline comuni. Densit&agrave; degli stati. Trasporto: da
    Drude a Sommerfeld e ricapitolazione delle equazioni di trasporto lineare della
    carica e del calore. Teoria del cristallo armonico e fononi. Propriet&agrave;
    ottiche dei semiconduttori e degli isolanti. Trasporto di carica nei
    semiconduttori intrinseci e drogati. Ricapitolazione del teorema

In [ ]:
# Esempio: come interrogare Course Catalogue per avere i dettagli di un corso di studi

anno      = 2024
codiceGDA = 10452

import requests
import textwrap

url = f"https://unipi.coursecatalogue.cineca.it/api/v1/corso/{anno}/{codiceGDA}"
response = requests.get(url)

if response.status_code == 200:

    data = response.json()
    print(f"Nome del CdS:        {data['des_it']}")
    print(f"Codice CdS:          {data['cdsCod']}")
    print(f"Id CdS (codice GDA): {data['cdsId']}")
    print(f"Anno accademico:     {data['aa']}")
    print(f"Ordinamento:         {data['ordinamento_aa']}")
    print(f"\nCurricula:\n")
    for idx, percorso in enumerate(data['percorsi']):
        print(f"{idx+1}. {percorso['des_it']:<30} [codice {percorso['pdsCod']:>2}]")

    print("\nLista codici esame:\n")
    codici = set()
    for percorso in data['percorsi']:
        for anno in percorso['anni']:
            if 'insegnamenti' in anno:
                for inss in anno['insegnamenti']:
                    for ins in inss:
                        for atts in inss['attivita']:
                            codici.add(atts['adCod'])
            else: # c'è un anno con direttamente le attività
                for atts in anno['attivita']:
                    codici.add(atts['adCod'])
    listone = " ".join(sorted(codici))
    for line in textwrap.wrap(listone, 80):
        print(f"{line}")

else:
    print(f"Errore nella richiesta: {response.status_code}")

Nome del CdS:        FISICA
Codice CdS:          WFI-LM
Id CdS (codice GDA): 10452
Anno accademico:     2024
Ordinamento:         2023

Curricula:

1. FISICA TEORICA                 [codice  7]
2. INTERAZIONI FONDAMENTALI       [codice  8]
3. FISICA MEDICA                  [codice 10]
4. ASTRONOMIA E ASTROFISICA       [codice 11]
5. GENERALE                       [codice 12]
6. FISICA DELLA MATERIA           [codice  9]

Lista codici esame:

0004B 0005B 0007B 0008B 0009B 0010B 0011B 0012B 0013B 0014B 0015B 0016B 0017B
0018B 0019B 0020B 0021B 0022B 0023B 0024B 0025B 0026B 0027B 043BB 063BB 080BB
084BB 088BB 091BB 092BB 098BB 104BB 107BB 111BB 124BB 125ZW 137BB 140BB 170BB
185BB 187BB 189BB 190BB 192BB 193BB 196BB 197BB 198BB 201BB 203BB 204BB 206BB
207BB 211BB 213BB 214BB 217BB 218BB 219BB 221BB 223BB 226BB 227BB 228BB 230BB
235BB 244CC 251BB 256BB 257BB 274BB 275BB 277BB 278BB 279BB 286BB 302BB 304BB
305BB 306BB 307BB 308BB 309BB 322BB 323BB 324BB 326BB 327BB 328BB 338BB 352BB
353BB 35